<a href="https://colab.research.google.com/github/netaamz/cloudTut6/blob/main/pythongame.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

creating the DB with questions and stufrnts

In [ ]:
!pip install firebase 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


insert questions

In [ ]:
from firebase import firebase
import firebase_admin

from firebase_admin import db
import json


# Initialize the app with the service account credentials
FBconn = firebase.FirebaseApplication('https://pythongame-a4fc4-default-rtdb.firebaseio.com',None)


# Define the questions in a dictionary with a level field
questions = {
    "question1": {
        "level": 1,
        "question": "What is the keyword used to define a function in Python?",
        "answers": ["def", "func", "define", "function"],
        "correct_answer": "def"
    },
    "question2": {
        "level": 1,
        "question": "Which of the following is a data type in Python?",
        "answers": ["Integer", "String", "Boolean", "All of the above"],
        "correct_answer": "All of the above"
    },
    "question3": {
        "level": 2,
        "question": "What is the result of 2 + 3 * 4?",
        "answers": ["20", "14", "24", "18"],
        "correct_answer": "14"
    },
    "question4": {
        "level": 2,
        "question": "Which of the following is NOT a Python built-in data type?",
        "answers": ["List", "Tuple", "Dictionary", "Stack"],
        "correct_answer": "Stack"
    },
    "question5": {
        "level": 3,
        "question": "What is the output of the following code? print('Hello', 'world!', sep='')",
        "answers": ["Hello, world!", "Hello world!", "Helloworld!", "None of the above"],
        "correct_answer": "Helloworld!"
    },
    "question6": {
        "level": 3,
        "question": "What is the correct syntax for a for loop in Python?",
        "answers": ["for i in range(10):", "for i = 0 to 9:", "for i in 1...10:", "for i = 1 until 10:"],
        "correct_answer": "for i in range(10):"
    },
    "question7": {
        "level": 4,
        "question": "What is the output of the following code? print(5 == '5')",
        "answers": ["True", "False", "TypeError", "SyntaxError"],
        "correct_answer": "False"
    },
    "question8": {
        "level": 4,
        "question": "What is the keyword used to indicate the end of a loop in Python?",
        "answers": ["break", "exit", "continue", "end"],
        "correct_answer": "break"
    },
    "question9": {
        "level": 5,
        "question": "Which of the following is a valid way to define a list in Python?",
        "answers": ["list1 = [1, 2, 3]", "list2 = (1, 2, 3)", "list3 = {1, 2, 3}", "list4 = '123'"],
        "correct_answer": "list1 = [1, 2, 3]"
    },
    "question10": {
        "level": 3,
        "question": "What is the output of the following code? print(len('Python'))",
        "answers": ["5", "6", "7", "None of the above"],
        "correct_answer": "6"
    }
}
result= FBconn.post('/python_questions/',questions)
# Push the questions dictionary to the Firebase database
#ref = db.reference('python_questions')
#ref.set(questions)

insert students 

In [ ]:
from firebase import firebase
import firebase_admin

from firebase_admin import db
import json


# Initialize the app with the service account credentials
FBconn = firebase.FirebaseApplication('https://pythongame-a4fc4-default-rtdb.firebaseio.com',None)

students = [
    {
        "id": "5678",
        "name": "Sarah Miller",
        "email": "sarah.miller@example.com",
        "total_score": 85,
        "level": 3,
        "history": [
            {
                "question_id": "question1",
                "is_correct": True
            },
            {
                "question_id": "question3",
                "is_correct": True
            },
            {
                "question_id": "question4",
                "is_correct": True
            },
            {
                "question_id": "question5",
                "is_correct": False
            },
            {
                "question_id": "question9",
                "is_correct": True
            }
        ]
    },
    {
        "id": "9012",
        "name": "David Cohen",
        "email": "david.cohen@example.com",
        "total_score": 75,
        "level": 2,
        "history": [
            {
                "question_id": "question2",
                "is_correct": False
            },
            {
                "question_id": "question3",
                "is_correct": True
            },
            {
                "question_id": "question4",
                "is_correct": True
            },
            {
                "question_id": "question6",
                "is_correct": False
            },
            {
                "question_id": "question10",
                "is_correct": False
            }
        ]
    },
    {
        "id": "3456",
        "name": "Rachel Levy",
        "email": "rachel.levy@example.com",
        "total_score": 90,
        "level": 3,
        "history": [
            {
                "question_id": "question1",
                "is_correct": True
            },
            {
                "question_id": "question2",
                "is_correct": False
            },
            {
                "question_id": "question3",
                "is_correct": True
            },
            {
                "question_id": "question5",
                "is_correct": True
            },
            {
                "question_id": "question7",
                "is_correct": True
            }
        ]
    },
    {
        "id": "7890",
        "name": "Daniel Goldstein",
        "email": "daniel.goldstein@example.com",
        "total_score": 65,
        "level": 1,
        "history": [
            {
                "question_id": "question2",
                "is_correct": False
            },
            {
                "question_id": "question4",
                "is_correct": False
            },
            {
                "question_id": "question6",
                "is_correct": True
            },
            {
                "question_id": "question7",
                "is_correct": False
            },
            {
                "question_id": "question10",
                "is_correct": False
            }
        ]
    }
]
result= FBconn.post('/users/',students)
# Push the questions dictionary to the Firebase database
#ref = db.reference('python_questions')
#ref.set(questions)

תבנית של HTML JS CSS 


In [8]:
import IPython
from google.colab import drive
drive.mount('/content/drive')

html_code = \
'''
<!DOCTYPE html>
<html>
<head>
   <link rel="stylesheet" type="text/css" href="https://drive.google.com/uc?id=1_at-FCZtgSJxIf55lVye2baR9SwjKjO8">
    <title>Quiz</title>
    <script src="https://drive.google.com/uc?export=view&id=1gtSP_dhC_RrCq8iPaylCY11NHw-NvDwH"></script>
</head>
<body onload="NextQuestion(0)">
    <main>
        <!-- creating a modal for when quiz ends -->
        <div class="modal-container" id="score-modal">
          
            <div class="modal-content-container">
              
                <h1>Congratulations, Quiz Completed.</h1>
              
                <div class="grade-details">
                    <p>Attempts : 10</p>
                    <p>Wrong Answers : <span id="wrong-answers"></span></p>
                    <p>Right Answers : <span id="right-answers"></span></p>
                    <p>Grade : <span id="grade-percentage"></span>%</p>
                    <p ><span id="remarks"></span></p>
                </div>
              
                <div class="modal-button-container">
                    <button onclick="closeScoreModal()">Continue</button>
                </div>
              
            </div>
        </div>

        <div class="game-quiz-container">
          
            <div class="game-details-container">
                <h1>Score : <span id="player-score"></span> / 10</h1>
                <h1> Question : <span id="question-number"></span> / 10</h1>
            </div>

            <div class="game-question-container">
                <h1 id="display-question"></h1>
            </div>

            <div class="game-options-container">
              
               <div class="modal-container" id="option-modal">
                 
                    <div class="modal-content-container">
                         <h1>Please Pick An Option</h1>
                      
                         <div class="modal-button-container">
                            <button onclick="closeOptionModal()">Continue</button>
                        </div>
                      
                    </div>
                 
               </div>
              
                <span>
                    <input type="radio" id="option-one" name="option" class="radio" value="optionA" />
                    <label for="option-one" class="option" id="option-one-label"></label>
                </span>
              

                <span>
                    <input type="radio" id="option-two" name="option" class="radio" value="optionB" />
                    <label for="option-two" class="option" id="option-two-label"></label>
                </span>
              

                <span>
                    <input type="radio" id="option-three" name="option" class="radio" value="optionC" />
                    <label for="option-three" class="option" id="option-three-label"></label>
                </span>
              

                <span>
                    <input type="radio" id="option-four" name="option" class="radio" value="optionD" />
                    <label for="option-four" class="option" id="option-four-label"></label>
                </span>


            </div>

            <div class="next-button-container">
                <button onclick="handleNextQuestion()">Next Question</button>
            </div>

        </div>
    </main>
    <script src="index.js"></script>
</body>
</html>

'''
display(IPython.display.HTML(html_code))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
